<a href="https://colab.research.google.com/github/brettmorrisonAO5ANNEX/Super-Resolution-Implementation/blob/main/SRCNN_Test.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image
from sklearn.model_selection import train_test_split
import tensorflow as tf
from tensorflow.keras.layers import Conv2D
from tensorflow.keras.models import Sequential
from tensorflow.keras.optimizers import Adam
from data import DIV2K

##Data Preprocessing

In [ ]:
from data import DIV2K

train_loader = DIV2K(scale=4,             # 2, 3, 4 or 8
                     downgrade='bicubic', # 'bicubic', 'unknown', 'mild' or 'difficult'
                     subset='train')      # Training dataset are images 001 - 800

# Create a tf.data.Dataset
train_ds = train_loader.dataset(batch_size=16,         # batch size as described in the EDSR and WDSR papers
                                random_transform=True, # random crop, flip, rotate as described in the EDSR paper
                                repeat_count=None)     # repeat iterating over training images indefinitely

# Iterate over LR/HR image pairs
# for lr, hr in train_ds:
    # ....

##Creating The Model

In [ ]:
# TODO

##Training & Evaluation

In [ ]:
# TODO